# COVID-19 Data Collection

Authors (in alphabetical order): Frederic Poitevin, Joao Rodrigues, Andrea Scaiewicz

This notebook pulls and merges data from several sources to create a standardized format for analysis.

In [ ]:
import datetime
import pathlib
import re

import numpy as np
import pandas as pd
import pycountry

In [ ]:
output_dir = pathlib.Path('..') / '..' / 'output'  # directory where final csv file is written to.

In [ ]:
pst_tz = datetime.timezone(datetime.timedelta(hours=-7))  # define the timezone

# Generate date stamps from 1/22/2020 until yesterday
start_date = datetime.datetime(2020, 1, 23).astimezone(pst_tz)  # offset for PST

today_date = datetime.datetime.now(datetime.timezone.utc).astimezone(pst_tz)
end_date = (today_date - datetime.timedelta(days=1))

dates_cols = [
    f'{d.month}/{d.day}/{str(d.year)[2:]}'  # date format should be e.g. 1/7/20 (July 1st, 2020)
    for d in pd.date_range(start_date, end_date)
]

__World Data__

First, we get raw data from JHU (source: https://github.com/CSSEGISandData/COVID-19/). JHU stores data for country-level data for confirmed and deaths in separate files. We need to merge these into one coherent file.

In [ ]:
def read_jhu_country_level_data(url):
    
    # Define which columns we want to read
    columns = {
        'Province/State' : str,
        'Country/Region': str
    }

    columns.update(
        {k: int for k in dates_cols}
    )
    
    df = pd.read_csv(
        url,
        usecols=list(columns.keys()),
        dtype=columns  # Specify dtype to save memory on load and check for bad values
    )
    
    df.rename(
        {
            'Country/Region': 'Country_Region',
            'Province/State': 'Province_State'
        },
        axis=1,
        inplace=True
    )
    return df

In [ ]:
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df_confirmed = read_jhu_country_level_data(confirmed_url)
df_confirmed.insert(loc=2, column='Case_Type', value='Confirmed')

In [ ]:
death_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df_deaths = read_jhu_country_level_data(death_url)
df_deaths.insert(loc=2, column='Case_Type', value='Deaths')

In [ ]:
assert len(df_deaths) == len(df_confirmed) # sanity check

In [ ]:
df_countries = df_confirmed.append(df_deaths)
df_countries.sort_values(by=['Country_Region', 'Province_State'], inplace=True)

__US State/County Data__

We get raw data from JHU (source: https://github.com/CSSEGISandData/COVID-19/). JHU stores data for county-level data for confirmed and deaths in separate files. We need to merge these into one file again.

In [ ]:
def read_jhu_US_county_level_data(url):
    
    # Define which columns we want to read
    columns = {
        'Province_State' : str,
        'Country_Region': str,
        'Admin2': str,
    }

    columns.update(
        {k: int for k in dates_cols}
    )
    
    df = pd.read_csv(
        url,
        usecols=list(columns.keys()),
        dtype=columns  # Specify dtype to save memory on load and check for bad values
    )
    
    df.rename(
        {
            'Admin2': 'County_Name',
        },
        axis=1,
        inplace=True
    )
    
    return df

In [ ]:
us_confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
df_us_confirmed = read_jhu_US_county_level_data(us_confirmed_url)
df_us_confirmed.insert(loc=4, column='Case_Type', value='Confirmed')

In [ ]:
us_death_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df_us_deaths = read_jhu_US_county_level_data(us_death_url)
df_us_deaths.insert(loc=4, column='Case_Type', value='Deaths')

In [ ]:
assert len(df_us_confirmed) == len(df_us_deaths)

In [ ]:
df_us_counties = df_us_confirmed.append(df_us_deaths)

__Merge JHU Datasets__

Now we need to merge JHU datasets, making sure the columns align properly.

In [ ]:
# Add extra columns in df_us_counties to df_countries
df_countries['County_Name'] = ''

In [ ]:
# Remove the total of the US from the world data
mask = df_countries['Country_Region'] == 'US'
df_countries = df_countries[~mask]

In [ ]:
df_jhu_combined = df_countries.append(df_us_counties)

In [ ]:
df_jhu_combined['Source'] = 'jhu'

__Italian Regional Data__

We get data for Italian Regions.

In [ ]:
url_italy = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
df_italy = pd.read_csv(url_italy)

to_keep = set(('deceduti', 'totale_casi', 'data', 'denominazione_regione'))
to_drop = [c for c in df_italy.columns if c not in to_keep]
df_italy.drop(
    columns=to_drop,
    inplace=True
)

# First, we translate the column labels and get rid of the unimportant ones.
# A region - _regione_ - is the equivalent of a Province in the JHU dataset.

df_italy.rename(
    columns={
        "data": "Date",
        "denominazione_regione": "Province/State",
    },
    inplace=True
)

df_italy.rename(
    columns={
        "Province/State": "Province_State",
        "totale_casi": "Confirmed",
        "deceduti": "Deaths"
    },
    inplace=True
)

The Italy dataframe is missing dates. We need to add the missing ones.

In [ ]:
# First we identify and match dates
raw_italy_dates = [
    t.replace(hour=0, minute=0, second=0)
    for t in pd.to_datetime(df_italy['Date'].unique())
]

# Add non-existing dates (from Jan 21st)
start_date = datetime.datetime(2020, 1, 22)
end_date = datetime.datetime.today() - datetime.timedelta(days=1)
missing_dates = [
    i for i in pd.date_range(start_date, end_date)
    if i not in raw_italy_dates
]

italy_dates = [
    f'{d.month}/{d.day}/{str(d.year)[2:]}'
    for d in missing_dates + raw_italy_dates
]

date_to_idx = {
    d: idx for idx, d in enumerate(italy_dates)
}

Now we need to flip the Italy dataframe, to match JHU format

In [ ]:
provinces = list(df_italy['Province_State'].unique())

datadict = {
    p: {
        'Confirmed': [0 for d in italy_dates],
        'Deaths': [0 for d in italy_dates]
    }
    for p in provinces
}  # province -> cases

for row_idx in range(len(df_italy)):
    date, province, deaths, confirmed = df_italy.iloc[row_idx].to_list()
    assert deaths <= confirmed  # sanity check
    
    d = pd.to_datetime(date)
    date_as_str = f'{d.month}/{d.day}/{str(d.year)[2:]}'
    date_idx = date_to_idx[date_as_str]
    
    datadict[province]['Confirmed'][date_idx] = confirmed
    datadict[province]['Deaths'][date_idx] = deaths

data_rows = [
    [p, 'Confirmed'] + datadict[p]['Confirmed']
    for p in datadict
]
data_rows.extend([
    [p, 'Deaths'] + datadict[p]['Deaths']
    for p in datadict
])

df_italy_T = pd.DataFrame.from_records(
    data_rows,
    columns=[
        'Province_State',
        'Case_Type',
    ] + italy_dates
)

df_italy_T.sort_values(['Province_State', 'Case_Type'], inplace=True)

In [ ]:
# Add Country_Region, County, and Source columns to match JHU
df_italy_T['Country_Region'] = 'Italy'
df_italy_T['County_Name'] = ''  # Empty
df_italy_T['Source'] = 'dpc-covid19-ita-province'

In [ ]:
# Remove the total of Italy from the world data
mask = df_jhu_combined['Country_Region'] == 'Italy'
df_jhu_combined = df_jhu_combined[~mask]

In [ ]:
df_combined = df_jhu_combined.append(df_italy_T)

__Standardize Names__

Country names have non-standard characters that may not work well as filenames. Let's standardize that.

In [ ]:
def sanitize_name(name):
    """Replaces all non-alphanumerical characters by underscores"""
    if not pd.isnull(name):
        return re.sub('[^A-Za-z0-9_]+', '_', name).strip('_')
    return name

In [ ]:
for colname in ['Country_Region', 'Province_State', 'County_Name']:
    col_safe_name = colname + '_Safe'
    df_combined[col_safe_name] = df_combined[colname].apply(sanitize_name)

__Final Preparations & Saving__

Add a timestamp column with last update date (UTC timezone).

In [ ]:
df_combined["Last_Update_Date"] = datetime.datetime.utcnow()

Sort dataframe by country, then province, then county.

In [ ]:
df_combined.sort_values(
    by=['Country_Region', 'Province_State', 'County_Name', 'Case_Type'],
    inplace=True
)

Write result dataframe to disk as a CSV file.

In [ ]:
df_combined.to_csv(
    str(output_dir / "Data_COVID-19_v2.csv"),  # Path to str
    index=False,
    columns=[
        "Country_Region",
        "Country_Region_Safe",
        "Province_State",
        "Province_State_Safe",
        "County_Name",
        "County_Name_Safe",
        "Case_Type",
        "Source",
        "Last_Update_Date"
    ] + dates_cols
)

__Aggregate Data at Country/State Level__

In [ ]:
groups = df_combined.groupby(
    [
        'Country_Region', 
        'Country_Region_Safe',
        'Case_Type',
        'Source',
        'Last_Update_Date'
    ]
)
df_bycountry = groups.sum().reset_index()

df_bycountry.to_csv(
    str(output_dir / "Data_COVID-19_v2_bycountry.csv"),  # Path to str
    index=False,
    columns=[
        "Country_Region",
        "Country_Region_Safe",
        "Case_Type",
        "Source",
        "Last_Update_Date"
    ] + dates_cols
)

In [ ]:
groups = df_combined.groupby(
    [
        'Country_Region', 
        'Country_Region_Safe',
        'Province_State',
        'Province_State_Safe',
        'County_Name',       
        'County_Name_Safe',
        'Case_Type',
        'Source',
        'Last_Update_Date'
    ]
)
df_bystate = groups.sum().reset_index()

df_bystate.to_csv(
    str(output_dir / "Data_COVID-19_v2_bystate.csv"),  # Path to str
    index=False,
    columns=[
        "Country_Region",
        "Country_Region_Safe",
        "Province_State",
        "Province_State_Safe",
        "County_Name",       
        "County_Name_Safe",
        "Case_Type",
        "Source",
        "Last_Update_Date"
    ] + dates_cols
)